In [49]:
import import_ipynb

from statistics import mean
from DataSet import *

In [50]:
net.to("cuda:0")
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()

In [51]:
for epoch in range(5):
    loss_log = []
    for x, y in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        o = net(x)
        
        loss = loss_f(o, y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        loss_log.append(loss.item())
        
    test_score = eval_net(net, test_loader, device="cuda:0")
    print(epoch, mean(loss_log), test_score, flush=True)

100%|██████████| 17579/17579 [04:51<00:00, 60.21it/s]


0 0.7801136108610883 0.6461886763572693


100%|██████████| 17579/17579 [05:30<00:00, 53.16it/s]


1 0.7777446113461974 0.6454557776451111


100%|██████████| 17579/17579 [10:43<00:00, 27.34it/s]


2 0.7769620873949906 0.6452344059944153


100%|██████████| 17579/17579 [11:27<00:00, 25.57it/s]


3 0.7767467676654015 0.6441600918769836


100%|██████████| 17579/17579 [11:11<00:00, 26.18it/s]


4 0.7765063144280592 0.6447762250900269


In [77]:
# Move Trained-Model to 'CPU'.
# Thankfully, I don't use VM(Virtual-Machine), but use CUDA,
# so it works smoothly.
net.to("cpu")

# Calculate evaluation of User1 of Movie10.
query = (1, 10)
print(query)


# Convert as 'int64' Tensor and add batch-dimension.
query = torch.tensor(query, dtype=torch.long).view(1, -1)

# Send to 'net'.
net(query)
print(df.dtypes)
print(net(query))

(1, 10)
userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object
tensor([3.3960], grad_fn=<MulBackward0>)


In [73]:
# Select User1 of Top-5-movies.
query = query.float()
query = torch.stack([
    torch.zeros(max_item).fill_(1),
    torch.arange(1, max_item+1)
], 1).float()

# 'scores' is a number of Top-k scores.
# 'indices' is a number of Top-k position,
# it means movieId.
scores, indices = torch.topk(net(query), 5)

RuntimeError: Expected object of scalar type float but got scalar type __int64 for sequence element 1.